<a href="https://colab.research.google.com/github/hanumaisyaah/Quiz2_BigData/blob/master/Big_Data_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [4]:
!pip install -q findspark

In [5]:
import findspark
findspark.init()

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
 .master("local")\
 .appName("Colab")\
 .config('spark.ui.port', '4050')\
 .getOrCreate()

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
!wget --continue https://raw.githubusercontent.com/dhanifudin/pyspark-demo/main/sample_books.json -O /tmp/sample_books.json

--2022-06-29 15:38:43--  https://raw.githubusercontent.com/dhanifudin/pyspark-demo/main/sample_books.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1565 (1.5K) [text/plain]
Saving to: ‘/tmp/sample_books.json’

/tmp/sample_books.j 100%[===================>]   1.53K  --.-KB/s    in 0s      

2022-06-29 15:38:43 (20.0 MB/s) - ‘/tmp/sample_books.json’ saved [1565/1565]



In [31]:
df = spark.read.json("/tmp/sample_books.json")

In [32]:
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)



In [33]:
df.select("title", "price", "year_written").show(5)

+----------------+-----+------------+
|           title|price|year_written|
+----------------+-----+------------+
|Northanger Abbey| 18.2|        1814|
|   War and Peace| 12.7|        1865|
|   Anna Karenina| 13.5|        1875|
|   Mrs. Dalloway| 25.0|        1925|
|       The Hours|12.35|        1999|
+----------------+-----+------------+
only showing top 5 rows



In [34]:
df_filtered = df.filter("year_written > 1950 AND price > 10 AND title IS NOT NULL")
df_filtered.select("title", "price", "year_written").show(50, False)

+-----------------------------+-----+------------+
|title                        |price|year_written|
+-----------------------------+-----+------------+
|The Hours                    |12.35|1999        |
|Harry Potter                 |19.95|2000        |
|One Hundred Years of Solitude|14.0 |1967        |
+-----------------------------+-----+------------+



In [45]:
from pyspark.sql import functions as F

In [47]:
maxValue = df_filtered.agg(F.max("price")).collect()[0][0]
print("maxValue: ",maxValue)
df_filtered.select("title","price").filter(df.price == maxValue).show(20, False)

maxValue:  19.95
+------------+-----+
|title       |price|
+------------+-----+
|Harry Potter|19.95|
+------------+-----+



In [51]:
minPriceeachAuthor = df.groupBy("author").agg(F.min("price")).show(20, False) 

+--------------------+----------+
|author              |min(price)|
+--------------------+----------+
|Tolstoy, Leo        |12.7      |
|Dickens, Charles    |5.75      |
|Marquez             |14.0      |
|Cunnningham, Michael|12.35     |
|Tolkien, J.R.       |27.45     |
|Woolf, Virginia     |25.0      |
|Shakespeare         |7.95      |
|Twain, Mark         |5.76      |
|Rowling, J.K.       |19.95     |
|Austen, Jane        |18.2      |
+--------------------+----------+



In [52]:
maxPriceeachAuthor = df.groupBy("author").agg(F.max("price")).show(20, False) 

+--------------------+----------+
|author              |max(price)|
+--------------------+----------+
|Tolstoy, Leo        |13.5      |
|Dickens, Charles    |5.75      |
|Marquez             |14.0      |
|Cunnningham, Michael|12.35     |
|Tolkien, J.R.       |27.45     |
|Woolf, Virginia     |29.0      |
|Shakespeare         |7.95      |
|Twain, Mark         |7.75      |
|Rowling, J.K.       |19.95     |
|Austen, Jane        |18.2      |
+--------------------+----------+



In [55]:
df_filtered = df.filter("price > 15.00 AND author == 'Austen, Jane' AND title IS NOT NULL")
df_filtered.select("title", "author", "price").show(50, False)

+----------------+------------+-----+
|title           |author      |price|
+----------------+------------+-----+
|Northanger Abbey|Austen, Jane|18.2 |
+----------------+------------+-----+

